In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 100)
user_data = pd.read_csv("/Users/kayleecho/Documents/data science projects/recommendation_system/synthetic_user_data.csv", sep=",")
print(user_data.shape)
user_data.head(10)

(1000, 23)


,user_id,investment_goal_long_term_growth,investment_goal_stable_income,investment_goal_balanced_approach,investment_goal_high_risk_high_reward,investment_goal_short_term_trading,risk_tolerance,holding_period_response,holding_period_reaction_strength,expected_return_score,product_preference_stocks,product_preference_etfs,product_preference_government_bonds,product_preference_corporate_bonds,product_preference_municipal_bonds,product_preference_savings_bonds,product_preference_derivatives,product_preference_alternative_investments,factor_weight_investment_goals,factor_weight_risk_tolerance,factor_weight_holding_period,factor_weight_return_score,factor_weight_product_preference
0,1,0.29,0.24,0.29,0.14,0.04,4.83,3.33,7.39,3.15,18.72,17.62,5.92,11.99,18.77,15.60,11.07,0.31,22.43,20.48,29.33,2.26,25.50
1,2,0.04,0.38,0.06,0.33,0.19,5.04,6.67,8.58,4.23,16.37,15.63,8.91,15.34,8.18,15.55,16.97,3.05,6.26,7.80,22.03,22.12,41.79
2,3,0.39,0.02,0.15,0.17,0.27,7.28,3.33,1.53,2.09,12.67,20.59,8.26,19.86,11.26,10.51,1.75,15.10,40.52,9.34,29.33,2.30,18.51
3,4,0.32,0.12,0.30,0.03,0.23,2.97,10.00,6.59,7.55,11.44,8.32,2.64,18.60,20.01,14.52,13.44,11.03,7.04,7.09,31.62,23.45,30.80
4,5,0.18,0.18,0.01,0.27,0.36,5.61,1.00,5.57,9.24,11.17,8.25,20.71,19.18,1.54,23.51,7.33,8.31,17.70,14.20,12.73,25.42,29.95
5,6,0.11,0.43,0.13,0.02,0.31,2.27,1.00,1.54,2.43,12.85,15.70,8.81,9.67,9.54,14.29,14.49,14.65,8.65,10.75,19.35,39.45,21.80
6,7,0.09,0.14,0.26,0.28,0.23,1.32,6.67,6.57,4.58,6.13,15.85,18.69,18.37,18.39,8.98,10.59,3.00,41.30,3.19,16.33,0.81,38.37
7,8,0.34,0.15,0.21,0.29,0.01,8.80,10.00,1.02,3.78,9.20,10.97,14.08,18.54,20.23,11.55,7.98,7.45,13.69,37.73,9.77,33.06,5.75
8,9,0.01,0.11,0.22,0.41,0.25,5.55,3.33,3.81,3.95,3.92,9.29,11.56,14.99,16.33,17.70,17.63,8.58,9.11,35.83,0.85,44.64,9.57
9,10,0.18,0.37,0.26,0.16,0.03,3.06,6.67,5.90,9.87,26.20,9.83,4.60,21.51,1.00,8.58,4.51,23.77,25.92,33.34,4.90,20.51,15.33


In [2]:
# the weights normally should add up to 1, so we are dividing each weight by 100 to match the scale!
w1 = user_data['factor_weight_investment_goals'] / 100  # Investment Goals weight
w2 = user_data['factor_weight_risk_tolerance'] / 100    # Risk Tolerance weight
w3 = user_data['factor_weight_holding_period'] / 100    # Holding Period weight
w4 = user_data['factor_weight_return_score'] / 100      # Return Score weight
w5 = user_data['factor_weight_product_preference'] / 100 # Product Preference weight

# if we are to use composite score, product preference score should also be in similar range compared to other categories in the composite score.
user_data["product_preference_stocks"] = user_data["product_preference_stocks"] / 100
user_data["product_preference_etfs"] = user_data["product_preference_etfs"] / 100
user_data["product_preference_government_bonds"] = user_data["product_preference_government_bonds"] / 100
user_data["product_preference_corporate_bonds"] = user_data["product_preference_corporate_bonds"] / 100
user_data["product_preference_municipal_bonds"] = user_data["product_preference_municipal_bonds"] / 100
user_data["product_preference_savings_bonds"] = user_data["product_preference_savings_bonds"] / 100
user_data["product_preference_derivatives"] = user_data["product_preference_derivatives"] / 100
user_data["product_preference_alternative_investments"] = user_data["product_preference_alternative_investments"] / 100


Score = w1 * Investment Goals + w2 * Risk Tolerance + w3 * Holding Period + w4 * Return Score (Expected) + w5 * Product Preference

In [3]:
# Investment goals score
investment_goal_cols = [
    'investment_goal_long_term_growth',
    'investment_goal_stable_income',
    'investment_goal_balanced_approach',
    'investment_goal_high_risk_high_reward',
    'investment_goal_short_term_trading'
]

# Risk tolerance score
risk_tolerance_score = user_data['risk_tolerance']

# Holding period score
holding_period_cols = [
    'holding_period_response',
    'holding_period_reaction_strength'
]

# Expected return score
return_score = user_data['expected_return_score']

# Product preference score
product_preference_cols = [
    'product_preference_stocks',
    'product_preference_etfs',
    'product_preference_government_bonds',
    'product_preference_corporate_bonds',
    'product_preference_municipal_bonds',
    'product_preference_savings_bonds',
    'product_preference_derivatives',
    'product_preference_alternative_investments'
]

In [4]:
# For those that have more than one score to consider, we create composite score for each category before multiplying weight and adding it to the larger composite score calculation
investment_goal_composite_score = 1 * user_data["investment_goal_long_term_growth"]  + 1 * user_data["investment_goal_stable_income"] + 2 * user_data["investment_goal_balanced_approach"] + 3 * user_data["investment_goal_high_risk_high_reward"] + 3 * user_data["investment_goal_short_term_trading"]

holding_period_composite_score = user_data["holding_period_response"] + user_data["holding_period_reaction_strength"]

product_preference_composite_score = 1 * user_data["product_preference_government_bonds"] + 2.5 * user_data["product_preference_corporate_bonds"] \
                                    + 4 * user_data["product_preference_stocks"] + 4 * user_data["product_preference_etfs"] \
                                    + 6 * user_data["product_preference_derivatives"] + 8 * user_data["product_preference_alternative_investments"]

1. calculating cosine similarity using five vector space instead of using one composite score.
2. Using one composite score by adding all five weighted vector space.

In [5]:
# This is calculating similarity using five vector space instead of using one composite score!
np.set_printoptions(threshold=np.inf)

import numpy as np

score_vector = np.column_stack((
    w1 * investment_goal_composite_score,
    w2 * risk_tolerance_score,
    w3 * holding_period_composite_score,
    w4 * return_score,
    w5 * product_preference_composite_score
))

score_vector.shape

(1000, 5)

In [6]:
from numpy.linalg import norm

new_users = score_vector[:20]
old_users = score_vector[20:]

cosine_similarities = {}

for i in range(len(new_users)):
    similarities = []
    for j in range(len(old_users)):
        cosine_similarity = np.dot(new_users[i], old_users[j]) / (norm(new_users[i]) * norm(old_users[j]))
        similarities.append(cosine_similarity)
    cosine_similarities[f"User_{i+1}"] = similarities

cosine_sim_df = pd.DataFrame.from_dict(cosine_similarities, orient='index')

print(cosine_sim_df.shape)
cosine_sim_df

(20, 980)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,...,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979
User_1,0.870835,0.821462,0.500509,0.990703,0.404884,0.925333,0.395929,0.398468,0.972754,0.795985,0.375756,0.947416,0.960808,0.994079,0.719632,0.727031,0.942337,0.911788,0.494428,0.791166,0.514668,0.836864,0.896111,0.917120,0.592869,0.961055,0.483729,0.870746,0.522013,0.931297,0.322924,0.228103,0.782875,0.874711,0.968878,0.969956,0.977590,0.753220,0.865115,0.672858,0.464421,0.979313,0.404700,0.957498,0.542549,0.958407,0.852229,0.675967,0.678532,0.277585,0.929273,0.989045,0.733981,0.984928,0.976638,0.794688,0.806395,0.258555,0.329472,0.692477,0.826588,0.593918,0.914373,0.735446,0.823935,0.869774,0.976274,0.677125,0.565470,0.921310,0.973641,0.771148,0.491364,0.984952,0.955888,0.957874,0.974150,0.343953,0.931969,0.642338,0.960456,0.973911,0.976306,0.280738,0.518761,0.495977,0.908801,0.616603,0.440140,0.533393,0.629604,0.987091,0.675099,0.941903,0.766010,0.964211,0.876158,0.342498,0.837662,0.978586,...,0.691020,0.973660,0.920495,0.989604,0.390849,0.968515,0.423641,0.841292,0.979177,0.807553,0.440461,0.765054,0.567762,0.270682,0.822082,0.606235,0.960551,0.417049,0.915015,0.992431,0.593702,0.489401,0.327439,0.825880,0.690842,0.362144,0.814527,0.392981,0.770945,0.807906,0.960909,0.793341,0.404304,0.963181,0.954577,0.971405,0.377117,0.210273,0.894785,0.401294,0.735500,0.894993,0.838664,0.710528,0.659855,0.876960,0.597296,0.899683,0.430139,0.530228,0.896504,0.903208,0.869902,0.975123,0.498157,0.613620,0.666595,0.712690,0.903808,0.935886,0.510237,0.979017,0.886124,0.883067,0.997513,0.981557,0.688540,0.759596,0.929017,0.667162,0.845483,0.809961,0.923976,0.842029,0.449955,0.687170,0.902808,0.981621,0.570520,0.912672,0.757756,0.783225,0.405045,0.546460,0.995427,0.566419,0.960927,0.693289,0.324865,0.562009,0.838123,0.884305,0.341935,0.410958,0.949284,0.780323,0.793960,0.226402,0.846226,0.981338
User_2,0.920025,0.806379,0.651027,0.972784,0.328799,0.966035,0.569226,0.321126,0.960718,0.761873,0.404027,0.966429,0.877512,0.940146,0.769647,0.876559,0.989441,0.973869,0.335386,0.881487,0.515640,0.864990,0.860049,0.906812,0.700618,0.967281,0.631052,0.916828,0.503362,0.934446,0.419291,0.467902,0.666698,0.823470,0.942591,0.858328,0.980814,0.749044,0.915953,0.661453,0.379071,0.975934,0.398462,0.939686,0.631715,0.932800,0.932101,0.830134,0.791982,0.502188,0.981705,0.949774,0.872868,0.978347,0.887003,0.884784,0.800354,0.470364,0.405897,0.573969,0.870604,0.713146,0.954918,0.745912,0.745361,0.941369,0.949063,0.834765,0.555721,0.971208,0.921931,0.632308,0.584199,0.915556,0.885015,0.954711,0.905022,0.451217,0.946749,0.770707,0.939712,0.938534,0.937421,0.316033,0.355836,0.496201,0.988995,0.460485,0.405153,0.425478,0.768163,0.903254,0.625298,0.983963,0.909905,0.949245,0.979632,0.400665,0.754521,0.947935,...,0.790367,0.978970,0.970820,0.959575,0.443447,0.969271,0.646402,0.914946,0.930711,0.750969,0.598145,0.590483,0.432804,0.510528,0.694563,0.667440,0.932368,0.614256,0.911678,0.937054,0.532554,0.412030,0.497651,0.730797,0.831283,0.518963,0.675338,0.406348,0.822098,0.729886,0.883254,0.864146,0.402399,0.983251,0.922049,0.985119,0.371826,0.361101,0.956795,0.453843,0.662647,0.872886,0.731527,0.791536,0.633901,0.897876,0.586731,0.966855,0.649466,0.406984,0.897706,0.828260,0.959923,0.977826,0.502877,0.609732,0.771398,0.756395,0.802328,0.997845,0.491496,0.965132,0.924547,0.963124,0.94683

In [7]:
cosine_sim_df_transposed = cosine_sim_df.T

cosine_sim_sorted = pd.DataFrame()

for i in cosine_sim_df_transposed.columns:
    sorted_indices = cosine_sim_df_transposed[i].sort_values(ascending=False).index
    cosine_sim_sorted[i] = cosine_sim_df_transposed[i].sort_values(ascending=False).values
    cosine_sim_sorted["old_user_id_for_" + i] = cosine_sim_df_transposed.index[sorted_indices].values

cosine_sim_sorted

,User_1,old_user_id_for_User_1,User_2,old_user_id_for_User_2,User_3,old_user_id_for_User_3,User_4,old_user_id_for_User_4,User_5,old_user_id_for_User_5,User_6,old_user_id_for_User_6,User_7,old_user_id_for_User_7,User_8,old_user_id_for_User_8,User_9,old_user_id_for_User_9,User_10,old_user_id_for_User_10,User_11,old_user_id_for_User_11,User_12,old_user_id_for_User_12,User_13,old_user_id_for_User_13,User_14,old_user_id_for_User_14,User_15,old_user_id_for_User_15,User_16,old_user_id_for_User_16,User_17,old_user_id_for_User_17,User_18,old_user_id_for_User_18,User_19,old_user_id_for_User_19,User_20,old_user_id_for_User_20
0,0.998761,306,0.997845,939,0.988626,974,0.998732,314,0.996032,733,0.987404,969,0.982354,782,0.987743,595,0.997858,336,0.994277,184,0.996292,753,0.988944,168,0.997749,557,0.995540,35,0.994082,98,0.996642,919,0.999655,291,0.973856,686,0.996949,952,0.995229,823
1,0.998412,740,0.995871,609,0.987349,433,0.998022,373,0.991898,2,0.985595,159,0.982149,207,0.987518,335,0.997201,83,0.994040,733,0.992713,889,0.988899,729,0.996997,305,0.991613,794,0.994026,494,0.994117,773,0.999196,416,0.971314,854,0.995686,230,0.991289,61
2,0.997513,944,0.995200,345,0.982805,450,0.997622,245,0.990377,878,0.984054,224,0.976895,475,0.986466,89,0.997138,177,0.993797,124,0.991662,636,0.986958,833,0.994729,145,0.991543,290,0.993752,273,0.970593,112,0.998941,883,0.967805,585,0.995137,447,0.990640,209
3,0.996156,191,0.993937,419,0.978745,459,0.997586,163,0.989484,749,0.983852,182,0.974311,340,0.980941,539,0.994756,791,0.990226,749,0.990827,728,0.986324,731,0.993630,87,0.991470,253,0.993699,951,0.967434,467,0.998772,474,0.967101,872,0.992448,513,0.990270,465
4,0.995902,338,0.993849,806,0.972997,601,0.997072,556,0.987967,680,0.982953,614,0.973481,564,0.980032,501,0.992974,683,0.988619,2,0.989868,667,0.985623,620,0.992185,259,0.989942,828,0.993171,331,0.966072,199,0.997729,13,0.965682,613,0.991527,383,0.987926,530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975,0.165100,763,0.258063,536,0.226626,431,0.208664,695,0.365041,317,0.370044,663,0.113053,122,0.243110,808,0.076969,914,0.317326,310,0.264643,170,0.347707,893,0.218632,388,0.159993,473,0.364411,473,0.099008,678,0.189714,228,0.324120,317,0.289407,681,0.250107,410
976,0.132693,228,0.253834,683,0.215709,763,0.195873,802,0.362888,310,0.351673,84,0.098779,83,0.225718,441,0.076051,271,0.311912,317,0.235358,378,0.346116,228,0.214830,893,0.148630,817,0.364174,388,0.091848,691,0.188134,598,0.324027,751,0.283890,388,0.247412,523
977,0.129983,817,0.245028,456,0.205875,473,0.174967,724,0.356833,914,0.331782,538,0.085253,622,0.220568,523,0.069257,803,0.308524,914,0.230053,441,0.343994,31,0.202925,31,0.142558,228,0.352361,654,0.091057,914,0.177838,388,0.316185,803,0.273992,654,0.244098,958
978,0.126701,388,0.244354,724,0.183988,654,0.170002,536,0.345499,751,0.327087,342,0.072718,683,0.182165,378,0.066851,317,0.295708,751,0.223791,523,0.302481,780,0.189799,228,0.135009,388,0.342342,228,0.070465,803,0.175093,817,0.314309,310,0.263889,228,0.222793,378


Here comes the second method logic which is using one composite score!

In [8]:
# Calculate composite score
composite_score = np.column_stack((
    w1 * investment_goal_composite_score + 
    w2 * risk_tolerance_score + 
    w3 * holding_period_composite_score + 
    w4 * return_score + 
    w5 * product_preference_composite_score 
))

composite_score

array([[ 5.21254025,  6.07971592,  3.57250075,  8.2698788 ,  5.152695  ,
         2.6148735 ,  4.10410577,  6.03729025,  4.30834332,  4.7289821 ,
         6.37867713,  2.85633308,  4.97721179,  3.36280781,  3.78086906,
         4.7605574 ,  6.14813212,  4.66634077,  3.74915752,  5.62599192,
         4.31151056,  3.85159492,  3.96246835,  6.14096624,  5.5283501 ,
         3.94531862,  4.83538989,  6.32539605,  5.6045844 ,  5.23407248,
         3.66286852,  4.89128602,  4.98844505,  7.2972434 ,  7.56792018,
         5.23987168,  4.66773972,  7.30225259,  4.67560808,  4.23176924,
         5.08035768,  6.59256063,  6.32141375,  5.76457853,  5.53877697,
         3.41601263,  4.54713368,  7.42049827,  5.14633554,  6.62555272,
         3.86856105,  5.47167393,  5.39351044,  6.0370312 , 10.16755445,
         3.59350276,  5.80388516,  3.7393728 ,  6.84655875,  5.7211988 ,
         3.38657958,  5.63391752,  4.18551264,  6.32402692,  3.72600632,
         3.80874954,  3.57882364,  5.548203  ,  6.0

In [9]:
from numpy.linalg import norm

composite_score = composite_score.T

new_users = composite_score[:20]
old_users = composite_score[20:]

cosine_similarities = {}

for i in range(len(new_users)):
    similarities = []
    for j in range(len(old_users)):
        cosine_similarity = np.dot(new_users[i], old_users[j]) / (norm(new_users[i]) * norm(old_users[j]))
        similarities.append(cosine_similarity)
    cosine_similarities[f"User_{i+1}"] = similarities

cosine_sim_df = pd.DataFrame.from_dict(cosine_similarities, orient='index')

print(cosine_sim_df.shape)
cosine_sim_df

(20, 980)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,...,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979
User_1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
User_2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
User_3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
User_4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
User_5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.

When there is only one dimensional vector space, cosine similarity becomes trival because it will cancel out to one.. Hence, if all our composite scores are positive, which is exactly our case, we will get 1.0 for every pair! So we are going to use different metric here. "Euclidean Distance" is what we're going to use. <br>
This will measure the straight line distance between the two vectors.

In [12]:
import numpy as np
import pandas as pd

new_users = composite_score[:20]
old_users = composite_score[20:]

euclidean_distances = {}

for i in range(len(new_users)):
    distances = []
    for j in range(len(old_users)):
        distance_array = np.abs(new_users[i] - old_users[j])
        distance = float(distance_array.item())
        disstance = round(distance, 5)
        distances.append(distance)
    euclidean_distances[f"User_{i+1}"] = distances

euclidean_dist_df = pd.DataFrame.from_dict(euclidean_distances, orient='index')

print("Euclidean distances shape:", euclidean_dist_df.shape)
euclidean_dist_df

Euclidean distances shape: (20, 980)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,...,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979
User_1,0.901030,1.360945,1.250072,0.928426,0.315810,1.267222,0.377150,1.112856,0.392044,0.021532,1.549672,0.321254,0.224095,2.084703,2.355380,0.027331,0.544801,2.089712,0.536932,0.980771,0.132183,1.380020,1.108874,0.552038,0.326237,1.796528,0.665407,2.207958,0.066205,1.413012,1.343979,0.259134,0.180970,0.824491,4.955014,1.619037,0.591345,1.473167,1.634019,0.508659,1.825961,0.421377,1.027028,1.111487,1.486534,1.403791,1.633717,0.335663,0.886636,0.832784,1.400299,1.598804,1.017572,1.268145,2.573770,0.505106,0.915811,0.107173,0.047584,0.702222,1.415648,1.823514,2.549033,1.648592,1.955562,5.085397,0.913464,0.757358,0.972778,1.636848,0.065695,0.774273,0.298371,0.593532,1.861552,1.155645,0.570879,0.859828,3.030303,1.074991,0.490226,2.378006,2.177469,1.428608,0.875373,1.371828,1.517202,1.028222,0.456882,0.225919,0.390885,0.941758,0.285720,0.100671,0.519802,0.505603,0.748697,0.101789,0.401152,2.033426,...,2.543943,0.417162,0.501781,0.240695,2.596559,0.594432,0.060240,0.397401,2.270751,2.966032,1.741961,1.681340,1.488494,0.263647,1.514690,1.683888,2.916680,1.221753,0.583151,1.199851,0.783340,1.366146,0.450435,1.848825,0.716759,0.988775,0.293520,2.386837,1.051397,0.148736,0.071102,1.195610,1.204050,0.859529,3.093224,0.029015,2.231458,0.046598,1.748602,1.782349,0.174772,0.973366,0.426814,1.659348,0.561561,0.338521,2.442525,1.129294,0.397609,1.843934,1.081810,0.891676,1.433734,0.740232,1.226910,1.479891,1.302959,1.704930,0.668021,1.005951,1.534498,0.992961,1.146574,0.762763,0.250765,0.343527,2.315652,2.290108,0.589929,2.852778,0.980967,1.205585,0.585582,0.314242,0.780740,1.854975,1.764267,0.822051,0.839261,1.073453,2.650677,0.754551,0.880282,1.832656,0.386031,0.895473,0.181425,1.432609,1.141119,1.069979,0.982968,1.402774,1.449055,1.648120,2.410241,0.054582,0.277165,1.493883,2.997821,0.834411
User_2,1.768205,2.228121,2.117248,0.061250,0.551366,2.134397,1.244326,0.245680,0.475132,0.845643,2.416847,1.188430,1.091271,1.217527,1.488204,0.839844,1.411976,1.222537,1.404108,1.847947,0.999358,0.512845,0.241698,0.315137,0.540939,2.663703,1.532582,1.340782,0.933380,0.545837,2.211155,0.608042,0.686205,0.042685,4.087839,2.486213,0.275831,2.340343,0.766843,0.358517,2.693136,0.445798,1.894203,0.244311,2.353710,2.270966,2.500892,0.531513,0.019460,1.699960,0.533123,0.731629,0.150396,0.400969,1.706594,0.362070,1.782987,0.760003,0.914760,0.164953,2.282823,0.956338,3.416209,2.515767,2.822738,4.218221,0.046288,1.624533,1.839954,0.769673,0.801481,0.092903,0.568804,1.460708,0.994376,0.288469,0.296297,0.007348,2.163127,0.207815,1.357402,1.510831,1.310293,0.561432,0.008198,0.504652,2.384377,1.895398,1.324057,1.093095,0.476291,0.074583,1.152896,0.967846,0.347373,1.372779,1.615873,0.968964,0.466024,1.166250,...,3.411119,0.450014,0.365395,0.626481,3.463734,1.461608,0.927416,0.469775,1.403575,2.098857,2.609137,0.814164,2.355669,1.130823,0.647514,2.551064,2.049505,2.088928,1.450327,0.332676,1.650516,2.233321,1.317610,2.716001,1.583935,1.855951,0.573655,3.254012,1.918573,0.718440,0.938277,2.062785,0.336874,1.726705,2.226048,0.838160,1.364283,0.820578,0.881426,2.649525,0.692404,1.840542,1.293989,2.526523,0.305614,0.528654,3.309701,0.262118,0.469567,2.711110,1.948985,0.024500,0.566559,1.607408,2.094086,0.612715,0.435783,2.572106,1.535197,0.138775,0.667322,1.860136,2.013750,1.629939,0.61641

It's sorted in ascending order since in euclidean distance, higher distance means the two values are further apart, indicating less similarity.

In [17]:
euclidean_df_transposed = euclidean_dist_df.T

euclidean_dist_sorted = pd.DataFrame()

for i in euclidean_df_transposed.columns:
    sorted_indices = euclidean_df_transposed[i].sort_values(ascending=True).index
    euclidean_dist_sorted[i] = euclidean_df_transposed[i].sort_values(ascending=True).values
    euclidean_dist_sorted["old_user_id_for_" + i] = euclidean_df_transposed.index[sorted_indices].values

euclidean_dist_sorted

,User_1,old_user_id_for_User_1,User_2,old_user_id_for_User_2,User_3,old_user_id_for_User_3,User_4,old_user_id_for_User_4,User_5,old_user_id_for_User_5,User_6,old_user_id_for_User_6,User_7,old_user_id_for_User_7,User_8,old_user_id_for_User_8,User_9,old_user_id_for_User_9,User_10,old_user_id_for_User_10,User_11,old_user_id_for_User_11,User_12,old_user_id_for_User_12,User_13,old_user_id_for_User_13,User_14,old_user_id_for_User_14,User_15,old_user_id_for_User_15,User_16,old_user_id_for_User_16,User_17,old_user_id_for_User_17,User_18,old_user_id_for_User_18,User_19,old_user_id_for_User_19,User_20,old_user_id_for_User_20
0,0.000551,570,0.001047,692,0.003098,209,0.002589,276,0.000395,886,0.001108,884,0.000035,345,0.000259,33,0.000195,736,0.003287,116,0.001326,544,0.008854,459,0.002880,272,0.000907,903,0.000319,224,0.001548,902,0.006166,91,0.001322,199,0.000369,598,0.003710,881
1,0.000823,175,0.002142,221,0.003914,850,0.027036,78,0.001288,292,0.046172,781,0.000440,318,0.000882,192,0.003167,0,0.003734,512,0.003137,753,0.016221,861,0.008592,706,0.004512,528,0.000754,383,0.004324,131,0.006781,680,0.001399,16,0.001871,185,0.007926,41
2,0.000949,877,0.002619,768,0.006323,46,0.035885,914,0.002545,395,0.048634,62,0.001933,708,0.001888,557,0.003390,817,0.005774,679,0.010492,75,0.017269,441,0.009409,89,0.005242,955,0.002198,120,0.004359,695,0.007166,3,0.009267,18,0.006045,298,0.009435,301
3,0.004161,813,0.007315,829,0.008080,973,0.042952,302,0.004591,628,0.053010,960,0.003319,378,0.002699,957,0.004598,109,0.006668,80,0.011150,137,0.030629,907,0.010467,206,0.005350,122,0.016023,60,0.004899,88,0.013669,818,0.010370,522,0.009785,37,0.011424,732
4,0.005935,314,0.007348,77,0.008500,822,0.059518,978,0.006359,28,0.053724,880,0.006669,854,0.005036,811,0.006558,475,0.007435,374,0.014955,815,0.036368,295,0.011233,12,0.005799,929,0.017384,972,0.007173,515,0.014979,417,0.013520,422,0.012189,748,0.012300,98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975,4.955014,34,4.218221,65,6.595054,34,5.910117,949,5.014859,34,7.552681,34,6.063449,34,4.260647,65,5.859211,34,5.438572,34,4.134046,148,7.311221,34,5.190343,34,6.804747,34,6.386685,34,5.406997,34,4.149805,65,5.501214,34,6.418397,34,4.541563,34
976,5.085397,65,4.237964,705,6.725436,65,5.942131,428,5.145242,65,7.683064,65,6.193831,65,4.263773,232,5.989594,65,5.568955,65,4.203559,682,7.441604,65,5.320725,65,6.935129,65,6.517068,65,5.537380,65,4.169548,705,5.631596,65,6.548780,65,4.671945,65
977,5.105140,705,4.306199,232,6.745179,705,6.094760,682,5.164985,705,7.702806,705,6.213574,705,4.280390,705,6.009336,705,5.588698,705,4.362261,312,7.461347,705,5.340468,705,6.954872,705,6.536811,705,5.557122,705,4.364591,148,5.651339,705,6.568522,705,4.691688,705
978,5.300183,148,4.433007,148,6.940222,148,6.253462,312,5.360028,148,7.897850,148,6.408617,148,4.475433,148,6.204380,148,5.783741,148,4.577572,327,7.656390,148,5.535511,148,7.149915,148,6.731854,148,5.752166,148,4.374615,232,5.846382,148,6.763566,148,4.886731,148
